# Welcome to ZebraTrack

How to get the best out of ZebraTrack
---
Please read the current section to ensure that your videos will be adequately analyzed by ZebraTrack.

 - ZebraTrack has only been trained on grayscale 480x640 resolution videos. Therefore, videos of any other format may not work with the program.
 - While video frame rate can be specified on input, the default is assumed to be 30Hz. Videos of a higher frame rate are accepted, whereas videos of a lower frame rate will not work.
 - Videos where the larva initiates a second swim pattern, unstimulated, will be analyzed but the results may include this second swim pattern and should be interpreted with caution.
 - Videos where the fish is occluded (by experimenter hair, for instance) are more likely to generate errors. However, ZebraTrack has been designed to effectively avoid distractor objects such as hands and forceps.
 - Videos that contain long periods of inactivity (i.e. where the larva is not swimming) are more likely to generate errors.
 - The IDEAL video for ZebraTrack is as follows:
	1. The experimenter stimulates the fish around 1 second into the video
	2. The larva executes only one swim pattern
	3. The video ends around 1 second after the larva stops swimming.
 - While ZebraTrack is an excellent tool, it is not perfect. You may find that some tracking data in the excel file contains lots of zeros in the swim distance column. If this is the case, feel free to delete these rows, which may appear either before or after the fish's swim pattern, to narrow down the tracking data.



# ZebraTrack How-To
Please follow the instructions below to use ZebraTrack, an automated tool for larval zebrafish tracking.

---

Please note the small arrow next to "Welcome to ZebraTrack". If you click this arrow, it will expand the current section and show you all the code. For usability, this section should remain collapsed. If you expand it by accident, simply click the arrow again to collapse the section.

---

Step 1: Make sure you have imported the ZebraTrack folder into your Google Drive's "Colab Notebooks" folder. If you do not have one, you can create one:


1.   Navigate to "My Drive" on the Google Drive website
2.   Press the plus button located in the top left of your screen, then select "new folder"
3. Name the new folder "Colab Notebooks"
4. Place the "ZebraTrack" folder under "Colab Notebooks"
5. Proceed with Step 2.


Step 2: Upload videos. If you have not already, this step will guide you through uploading videos so that you may run ZebraTrack on these videos.

1. To use ZebraTrack, you must upload at least one black and white, 480x640 resolution .avi video to ZebraTrack's "videos" folder. The easiest way to do so is to drag and drop a folder into ZebraTrack's Google Drive folder, and rename this folder "videos".
2. N.B. ZebraTrack can only analyze ONE TREATMENT GROUP at a time. This is because it places all your videos into one excel file. It will not distinguish videos based on their name. That is, if you have four treatment groups to analyze, you must run ZebraTrack four times; e.g., the first time you will upload only WT videos, then mutant videos, etc.


Step 3: Run the hidden cell below by clicking the play button at the bottom left of this text. This will run all the necessary code for you to use ZebraTrack in the next step.

1. If you have never used Google Colaboratory before, you will need to grant it access to your google drive. When you press run on the "Setup" cell, it will prompt you to connect to Google Drive.
2. You must give Google Colaboratory full access to your Google Drive so that it can read and write files stored in this location. If this step is not completed, then the program will not work.
3. N.B. Any time you change the contents of your Google Drive, you MUST run the setup cell again in order to ensure that ZebraTrack has access to your latest files.
4. Proceed with Step 4.


Step 4: Press run on the cell under "Run ZebraTrack" below. If all goes well, you should see lines of text rapidly being printed out. Once the program has finished running, you will find an excel file, titled "videos_results.xlsx" in your "videos" folder in Google Drive.

1. The average run time for the analysis of 20 videos is roughly 10 minutes.
2. ZebraTrack also outputs a folder named "predictions", which contains csv files. Each csv file represents the tracking data for a single larva in an ImageJ friendly format if you wish to make changes to or visualize the tracking data post-hoc. (N.B. these csvs are formatted to be used with ImageJ's "manual tracking" plugin)
3. If you would like to run ZebraTrack again, simply empty the videos folder, and repeat starting from Step 2.

In [23]:
!pip install ultralytics
!pip install xlsxwriter
!pip install OneEuroFilter --upgrade

import os
import re
import glob
import cv2
import csv
import numpy as np
import pandas as pd
import shutil


from ultralytics import YOLO
from collections import deque
from google.colab import drive
from OneEuroFilter import OneEuroFilter
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter


drive.mount("/content/drive", force_remount=True)


def list_avi_files(folder_path: str):
    """ Lists all .avi files in the given folder.

    Args:
      folder_path (str): A string specifying the path to the desired video folder

    Returns:
      list[str]: A list of strings, each representing a complete filepath to a video.
    """
    try:
        # List all files in the folder
        files = os.listdir(folder_path)
        # Filter out only .avi files
        avi_files = [file for file in files if file.endswith('.avi')]
        return avi_files
    except Exception as e:
        return str(e)


def video_to_predictions(video_directory: str, video_filename: str, training_run: int, use_last: bool):
    """ Provides predictions for larval zebrafish location in each frame of a specified video.

    Takes one video_name (str), one training_run (int) and use_last (bool) as inputs. Uses video_dir to locate a video to use,
    and then runs the model (specified by the training run number and use_last) on this video.
    Then uses the bounding box created by the model to refine tracking, finding the fish's head within the bounding box.
    These tracking data are then appended to a csv file which is written into a temporary folder within the video_directory.

    Args:
      video_directory (str): Path to the video folder
      video_filename (str): Name of the specific video within the video_directory
      training_run (int): Number specifying which version of the model should be used !!!~--- MAY BE OBSOLETE IN FINAL VERSION ---~!!!
      use_last (bool): Boolean specifying which version of the model should be used !!!~--- MAY BE OBSOLETE IN FINAL VERSION ---~!!!
    """
    # Extract the name of the file w/o the extension
    video_basename = os.path.splitext(os.path.basename(video_filename))[0]

    # Define the video filepath and open it
    video_filepath = os.path.join(video_directory, video_filename)

    cap = cv2.VideoCapture(video_filepath)
    ret, frame = cap.read()
    H, W, _ = frame.shape

    # Pre-defined box for searching for the central dot
    box_size = 100
    box_half = box_size // 2
    frame_center_x, frame_center_y = W // 2, H // 2
    search_box_x1, search_box_y1 = frame_center_x - box_half, frame_center_y - box_half
    search_box_x2, search_box_y2 = frame_center_x + box_half, frame_center_y + box_half

    # To store central dot positions across frames
    dot_positions = deque(maxlen=10)

    while ret:
        # Crop the frame to the search box
        search_box = frame[search_box_y1:search_box_y2, search_box_x1:search_box_x2]

        # Convert to grayscale if needed
        if len(search_box.shape) == 3:
            search_box = cv2.cvtColor(search_box, cv2.COLOR_BGR2GRAY)

        # Find the darkest pixel in the search box
        min_val, _, min_loc, _ = cv2.minMaxLoc(search_box)
        dot_x, dot_y = min_loc  # Local coordinates in the search box

        # Adjust to global frame coordinates
        global_dot_x = search_box_x1 + dot_x
        global_dot_y = search_box_y1 + dot_y
        dot_positions.append((global_dot_x, global_dot_y))

        # Read the next frame
        ret, frame = cap.read()

    # Release the video capture
    cap.release()

    # Calculate the average location of the central dot
    avg_dot_x = int(np.mean([pos[0] for pos in dot_positions]))
    avg_dot_y = int(np.mean([pos[1] for pos in dot_positions]))
    dot_radius = 2  # Define a radius around the central dot

    # Determine which model version to use
    best_or_last = 'last' if use_last else 'best'
    model_path = os.path.join('runs', 'detect', f'train{str(training_run)}', 'weights', f'{best_or_last}.pt')
    abs_model_path = os.path.join('drive/MyDrive/Colab Notebooks/ZebraTrack',  model_path)
    print(abs_model_path)

    # Load the YOLO model
    model = YOLO(abs_model_path)

    # Define threshold values for finding the fish head
    threshold = 0.2
    threshold_darkness = 140

    # Initialize list for storing tracking data
    results_data = []
    results_frame_number = 1

    cap = cv2.VideoCapture(video_filepath)
    ret, frame = cap.read()
    while ret:

        results = model(frame)[0]

        results_list = results.boxes.data.tolist()

        if len(results_list) == 0:
            results_data.append([str(results_frame_number), '1', np.nan, np.nan, '0'])
        else:
            x1, y1, x2, y2, confidence, class_id = results_list[0]

            # Convert values to integer
            x1 = round(float(x1))
            y1 = round(float(y1))
            x2 = round(float(x2))
            y2 = round(float(y2))

            if (float(confidence) > threshold):
                # Ensure the frame is grayscale before finding the darkest pixel
                if len(frame.shape) == 3:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Adjust the bounding box coordinates to expand the region, accounting for bboxes that don't fully enclose the fish
                x1_large = max(0, x1 - 4)
                y1_large = max(0, y1 - 4)
                x2_large = min(W, x2 + 4)
                y2_large = min(H, y2 + 4)
                bbox_region = frame[max(0, y1_large):min(H, y2_large), max(0, x1_large):min(W, x2_large)]

                # Check if central dot is inside the bounding box
                if ((x1_large - dot_radius) <= avg_dot_x <= (x2_large + dot_radius)) and ((y1_large - dot_radius) <= avg_dot_y <= (y2_large + dot_radius)):

                    # Apply dilation to reduce the number of black pixels in the image
                    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
                    frame_dilated = cv2.dilate(frame, kernel)

                    # Threshold to isolate dark objects in the entire frame
                    _, binary_frame = cv2.threshold(frame_dilated, threshold_darkness, 255, cv2.THRESH_BINARY_INV)

                    # Detect connected components
                    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_frame)

                    # Create a mask to exclude forceps
                    forceps_mask = np.zeros_like(binary_frame, dtype=np.uint8)

                    for i in range(1, num_labels):  # Skip the background (label 0)
                        area = stats[i, cv2.CC_STAT_AREA]
                        x, y, w, h = stats[i, cv2.CC_STAT_LEFT], stats[i, cv2.CC_STAT_TOP], stats[i, cv2.CC_STAT_WIDTH], stats[i, cv2.CC_STAT_HEIGHT]

                        # Filter based on size
                        if area > 100:
                            # Mask this component
                            component_mask = (labels == i).astype(np.uint8) * 255
                            forceps_mask = cv2.bitwise_or(forceps_mask, component_mask)

                    # Dilate the mask
                    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
                    dilated_forceps_mask = cv2.dilate(forceps_mask, kernel)

                    # Exclude forceps from the frame
                    frame_no_forceps = cv2.bitwise_and(frame, cv2.bitwise_not(dilated_forceps_mask))

                    # Take all pixels = 0 and invert them to white
                    frame_no_forceps[dilated_forceps_mask > 0] = 255

                    # Create a bounding box region with the forceps mask
                    bbox_masked = frame_no_forceps[max(0, y1_large):min(H, y2_large), max(0, x1_large):min(W, x2_large)]

                    # Only find the fish head if no forceps pixels are within the frame
                    if not np.any(bbox_masked == 255):
                        # Mask the central dot region in the bounding box
                        dot_x_local = avg_dot_x - x1_large
                        dot_y_local = avg_dot_y - y1_large

                        # Mask the central dot region by replacing it with white pixels
                        cv2.circle(bbox_masked, (dot_x_local, dot_y_local), dot_radius, 255, thickness=-1)

                        # Find the darkest pixel excluding the central dot
                        min_val, _, min_loc, _ = cv2.minMaxLoc(bbox_masked)

                        # Check if the darkest point is sufficiently dark
                        if min_val > threshold_darkness:
                            print("defaulting to central dot")
                        else:
                            x, y = min_loc
                            x += x1_large  # Adjust to the frame's coordinate system
                            y += y1_large
                            #print(f"The minimum value in frame {results_frame_number} is: {min_val}")
                    else:
                        # When the forceps are in the box, default to the center
                        x = int((x1 + x2)/2)
                        y = int((y1 + y2)/2)

                else:
                    # If the central dot is outside the bounding box, proceed normally
                    min_val, _, min_loc, _ = cv2.minMaxLoc(bbox_region)
                    x, y = min_loc
                    x += x1_large  # Adjust to the frame's coordinate system
                    y += y1_large

                if len(frame.shape) == 2:
                    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)

                # Append the x,y coordinates of the fish's head to the results
                results_data.append([results_frame_number, '1', x, y, confidence])

            else:
                # If confidence is not above threshold, append an empty row to signal failed detection
                results_data.append([str(results_frame_number), '1', np.nan, np.nan, '0'])

        ret, frame = cap.read()
        results_frame_number += 1

    # Create the output directory
    os.makedirs(f'{video_directory}/predicted_csvs/', exist_ok=True)
    csv_filename = f'{video_basename}.csv'
    csv_path = os.path.join(f'{video_directory}/predicted_csvs/', csv_filename)

    # Create the CSV file
    with open(csv_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Frame Number', 'Object Number', 'X', 'Y', 'Confidence Score'])

        for row in results_data:
            csvwriter.writerow(row)

    print(f'Saved {csv_filename}')

    cap.release()
    cv2.destroyAllWindows()

def videos_to_predictions(video_directory: str, training_run: int, use_last: bool):
    """ Runs video_to_predictions on all .avi files in a folder

    Args:
      video_directory (str): Path to the video folder
      training_run (int): Number specifying which version of the model should be used !!!~--- MAY BE OBSOLETE IN FINAL VERSION ---~!!!
      use_last (bool): Boolean specifying which version of the model should be used !!!~--- MAY BE OBSOLETE IN FINAL VERSION ---~!!!
    """

    for file in list_avi_files(video_directory):
      video_to_predictions(video_directory, file, 30, False)

def distance(x1, y1, x2, y2):
    """ Calculates the distance between 2 two-dimensional cartesian points

    Args:
      x1 (num): first x coordinate
      y1 (num) first y coordinate
      x2 (num): second x coordinate
      y2 (num): second y coordinate

    Returns:
      float: The distance between the 2 points
    """
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def correct_detections(df, xy_cal, max_velocity=300, max_distance=10, initial_position=(320, 240)):
    """ Corrects multiple detections in a csv file containing frame-by-frame predicted tracking data.

    Based on estimates for 2-day-old larval zebrafish maximum velocity, maximum distance travelled in 1/30 of a second, and the rough initial
    position of the larval zebrafish, determines which of 2 double detections are most likely to be correct. Often, the highest confidence prediction
    is correct, but this function has extra failsafes in cases where both predictions have low confidence.

    Args:
      df (pd.DataFrame): Pandas dataframe containing all the predicted tracking data for a single video
      xy_cal (float): The calibration value which corresponds to the number of millimetres in a pixel of video
      max_velocity (int): The maximum possible velocity for a larval zebrafish
      max_distance (int): The maximum distance a larval zebrafish can swim in 1/30 of a second
      initial_position (tuple): The center coordinates in a 640x480 frame of video

    Returns:
      pd.DataFrame: The corrected dataframe, in which all double detections (frames for which there is more than one zebrafish
      detected) have been rectified
    """
    # Convert initial position from pixels to millimeters
    previous_x, previous_y = initial_position[0] * xy_cal, initial_position[1] * xy_cal
    corrected_rows = []

    for frame in range(1, df['Frame Number'].max() + 1):
        frame_data = df[df['Frame Number'] == frame].copy()

        if frame_data.empty:
            continue

        best_row = None
        best_distance = float('inf')

        for _, row in frame_data.iterrows():
            # Convert x and y from pixels to millimeters
            x, y, confidence = row['X'] * xy_cal, row['Y'] * xy_cal, row['Confidence Score']
            dist = distance(previous_x, previous_y, x, y)

            if dist <= max_distance and dist < best_distance:
                best_distance = dist
                best_row = row

        if best_row is None:
            frame_data['Distance'] = frame_data.apply(
                lambda r: distance(previous_x, previous_y, r['X'] * xy_cal, r['Y'] * xy_cal), axis=1
                )
            if frame_data['Distance'].notna().any():
                best_row = frame_data.loc[frame_data['Distance'].idxmin()]
            elif len(frame_data) == 1:
                # Add the only available row, even if it's full of NaNs
                best_row = frame_data.iloc[0]

        if best_row is not None:
            corrected_rows.append(best_row)
            previous_x, previous_y = best_row['X'] * xy_cal, best_row['Y'] * xy_cal

    corrected_df = pd.DataFrame(corrected_rows)

    if 'Distance' in corrected_df.columns:
      corrected_df.drop('Distance', axis=1, inplace=True)

    return corrected_df

def remove_leading_trailing_nans(df):
    """ Removes leading and trailing np.nan values from the predicted tracking data.

    When predicted tracking data which contains np.nan values in columns 3 (x data) and 4 (y data),
    this function drops the np.nan containing rows to facilitate later processing. This must be done so
    that linear interpolation can be performed later in the correction stages.

    Args:
      df (pd.DataFrame): The dataframe containing the predicted tracking data

    Returns:
      pd.Dataframe: The df with all leading and trailing nan values removed.
    """
    # Drop leading rows with NaN values in columns 3 and 4
    while pd.isna(df.iloc[0, 2]) or pd.isna(df.iloc[0, 3]):
        df = df.iloc[1:]

    # Drop trailing rows with NaN values in columns 3 and 4
    while pd.isna(df.iloc[-1, 2]) or pd.isna(df.iloc[-1, 3]):
        df = df.iloc[:-1]

    return df.reset_index(drop=True)

def sanity_check(df, xy_cal, max_distance=10, distance_increment=2):
    """ Ensures no unrealistic distances are present in the predicted tracking data.

    Processes the DataFrame by replacing x and y values with NaN if the distance
    from the previous row to the current row exceeds 10mm in real space or if the velocity exceeds 300mm/s.

    Args:
        df (pd.DataFrame): DataFrame with at least 5 columns. x and y values are in columns 2 and 3 (0-indexed).
        xy_cal (float): Calibration factor to convert pixel values to real space.

    Returns:
        pd.DataFrame: Modified DataFrame with updated x and y values.
    """
    df = df.reset_index(drop=True)  # Reset the index to ensure continuous integer indexing
    for i in range(1, len(df)):
        if pd.isna(df.loc[i, 'X']) or pd.isna(df.loc[i, 'Y']):
            continue

        previous_index = i - 1
        nan_count = 0
        while previous_index >= 0 and (pd.isna(df.loc[previous_index, 'X']) or pd.isna(df.loc[previous_index, 'Y'])):
            previous_index -= 1
            nan_count += 1

        if previous_index < 0:
            continue

        x1, y1 = df.loc[previous_index, ['X', 'Y']] * xy_cal
        x2, y2 = df.loc[i, ['X', 'Y']] * xy_cal
        max_distance_adjusted = max_distance + nan_count * distance_increment
        distance_traveled = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

        if distance_traveled > max_distance_adjusted:
            df.loc[i, ['X', 'Y']] = np.nan

    return df

def interpolate_consecutive_nans(df):
    """ Interpolates NaN values using linear interpolation.

    If there are too many consecutive NaNs, uses the last valid value. This is done to fill np.nan values with useful
    data for further analysis. Since NaNs only appear rarely in the prediction data, and when they do appear,
    they rarely appear in large consecutive sequences, linear interpolation is an adequate estimate of fish movement in
    these brief periods.

    Args:
        df (pd.DataFrame): DataFrame with NaN values to interpolate.

    Returns:
        pd.DataFrame: Modified DataFrame with interpolated values.
    """
    max_consecutive_nans = 5  # Define a threshold for maximum consecutive NaNs

    # Interpolate NaNs
    df.interpolate(axis=0, inplace=True)

    # Handle consecutive NaNs by using the last valid value
    for col in [2, 3]:  # x and y columns
        is_nan = df.iloc[:, col].isna()
        for i in range(len(df)):
            if is_nan[i]:
                start_idx = i
                while i < len(df) and is_nan[i]:
                    i += 1
                if (i - start_idx) > max_consecutive_nans:
                    df.iloc[start_idx:i, col] = df.iloc[start_idx - 1, col]

    return df

def correct_csv_files(directory, xy_cal):
    """ Corrects all prediction errors in csv files of a directory

    Deletes all multiple detections in favour of the highest confidence detection, and fills in all missing detections by linear interpolation.

    Args:
      directory (str): Path to the directory
      xy_cal (float): The calibration value which corresponds to the number of millimetres in a pixel of video
    """
    corrected_dir = os.path.join(directory, "corrected_csvs")
    os.makedirs(corrected_dir, exist_ok=True)

    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)

            # remove leading and trailing failed detections
            df = remove_leading_trailing_nans(df)

            # correct multiple detections
            df = correct_detections(df, xy_cal)

            # correct erroneous detections by replacing the erroneous x and y values by np.nan
            df = sanity_check(df, xy_cal)

            # correct failed detections values by linear interpolation
            df = interpolate_consecutive_nans(df)

            # Return corrected df to a csv
            filepath_out = os.path.join(corrected_dir, filename)
            df.to_csv(filepath_out, index=False)

def calculate_velocity(x, y, frame_rate=30):
    """ Calculates the velocity between each consecutive (x, y) point.

    Args:
      x (list or np.array): X position data.
      y (list or np.array): Y position data.
      frame_rate (float): Frame rate of the data (frames per second).

    Returns:
      velocity (np.array): Velocity at each frame, calculated as the distance between consecutive points.
    """
    # Calculate the distances between consecutive points
    distances = np.sqrt(np.diff(x)**2 + np.diff(y)**2)

    # Convert distances to velocities (velocity = distance / time)
    # Time between frames is 1 / frame_rate
    velocities = distances * frame_rate

    return velocities

def filter_position_data(x_data, y_data, frame_rate, fcmin=5, beta=0, dc=1):
    """ Filters x and y position data using the OneEuroFilter.

    Args:
      x_data (list or np.array): The x position data.
      y_data (list or np.array): The y position data.
      freq (float): Frequency of the data in Hz (sampling rate). Default is 30 Hz.
      min_cutoff (float): Minimum cutoff frequency. Default is 1.0.
      beta (float): Speed coefficient. Adjusts the filter’s sensitivity to speed. Default is 0.0.
      d_cutoff (float): Cutoff frequency for the derivative of the signal. Default is 1.0.

    Returns:
      filtered_x (np.array): Filtered x position data.
      filtered_y (np.array): Filtered y position data.
    """

    # Create OneEuroFilter
    config = {
      'freq': 30,       # Hz
      'mincutoff': fcmin,  # Hz
      'beta': beta,
      'dcutoff': dc
      }

    f_x = OneEuroFilter(**config)
    f_y = OneEuroFilter(**config)

    filtered_x = []
    filtered_y = []

    # Apply the filter to the position data
    for t, (x, y) in enumerate(zip(x_data, y_data)):
        time = t / frame_rate  # Convert index to time based on the sampling rate
        filtered_x.append(f_x(x, time))
        filtered_y.append(f_y(y, time))

    return np.array(filtered_x), np.array(filtered_y)


def find_start_end_rows(df, x_pos, y_pos, frame_rate):
    """ Finds the start and end of the swim pattern within the predicted tracking data.

    Find the start and end row indices based on positional data filtering and subsequent analysis of velocity based on the filtered position data.

    Args:
      df (pd.DataFrame): The dataframe to analyze.
      x_pos, y_pos (pd.Series): filtered x and y positional data.

    Returns:
      tuple: A tuple containing the start row index and the end row index.
    """
    filtered_x, filtered_y = filter_position_data(df.loc[:, x_pos].to_numpy(), df.loc[:, y_pos].to_numpy(), frame_rate)

    velocity_filtered_pos = calculate_velocity(filtered_x, filtered_y, frame_rate)

    # Calculate velocity based off raw position data
    speed = calculate_velocity(filtered_x, filtered_y, frame_rate)

    # Prepend a 0 for the first frame (no movement before the first frame)
    speed = np.insert(speed, 0, 0)

    # Convert array to series to add to df
    df['Speed'] = pd.Series(speed)

    start_row = None
    end_row = None
    start_spd_thresh = 30
    end_spd_thresh = 15
    lo_spd_frames = 5

    # Find the start row, by looking for the SECOND frame in which velocity exceeds the threshold
    counter = 0
    for i in range(len(df)):
        if (df.loc[i, 'Speed'] >= start_spd_thresh) and (counter > 0):
            start_row = i - 1
            break
        elif (df.loc[i, 'Speed'] >= start_spd_thresh):
            counter += 1


    # If start_row is still None, it means no value >= 20 was found
    if start_row is None:
        return (-1,-1)  # -1 indicates the function failed

    # Find end row by iterating in reverse order over the filtered velocity values
    for i in range(len(df) - 1, start_row + 3, -1):
        if speed[i] >= end_spd_thresh:
            end_row = i
            break

    # If end_row is still None, it means no consistent low-velocity frames were found
    if end_row is None:
        end_row = len(df) - 1

    return start_row-1, end_row-1

def extract_metrics_from_predictions(xy_cal, frame_rate, csv_path, vid_num):
    """ Extracts only the useful tracking data from a predicted tracking csv file.

    Uses the start frame and end frame to create a new csv containing only the frames of interest. Performs calculations to obtain desired
    swim metrics that summarize the fish's swim time.

    Args:
      xy_cal (float): Calibration factor to convert pixel values to real space.
      frame_rate (int): Specifies the sampling rate so that swim duration and speed can be evaluated
    """
    df = pd.read_csv(csv_path)

    # Create output CSV path
    new_csv_path = os.path.split(csv_path)[0] + '/formatted_data/' + os.path.split(csv_path)[1]
    os.makedirs(os.path.split(new_csv_path)[0], exist_ok=True)

    # Define a helper function so that we can use the .apply() method on the xy columns
    def convert_to_real_space(cell):
        new_cell = cell * xy_cal
        return new_cell

    def convert_to_pixel_space(cell):
        new_cell = cell / xy_cal
        return new_cell

    # Create real space X and Y values in two new columns
    df['X_real'] = df['X'].map(convert_to_real_space)
    df['Y_real'] = df['Y'].map(convert_to_real_space)

    # Find the rows of interest, i.e. those in which the fish is moving
    start_row, end_row = find_start_end_rows(df, 'X_real', 'Y_real', frame_rate)

    if start_row == -1:
      print(f"No movement detected in video #{vid_num}!")
      return

    # Create the distance column
    df['distance_since_last_frame'] = np.nan  # Initialize the new column with NaN
    df.loc[1:, 'distance_since_last_frame'] = np.sqrt((df.loc[1:, 'X_real'] - df['X_real'].shift(1).iloc[1:])**2 + (df.loc[1:, 'Y_real'] - df['Y_real'].shift(1).iloc[1:])**2)

    # Create subset_df which contains only the frames during which the fish is moving
    subset_df = df.iloc[start_row:end_row+1]
    subset_df = subset_df.copy()

    # Create normalized X and Y rows (which are used to create figures by hand after data processing)
    subset_df.loc[:, 'X_norm'] = subset_df['X_real'] - subset_df.loc[start_row, 'X_real']
    subset_df.loc[:, 'Y_norm'] = subset_df['Y_real'] - subset_df.loc[start_row, 'Y_real']

    # Calculate the values for the metric column
    swim_duration = (subset_df['Frame Number'].iloc[-1] - subset_df['Frame Number'].iloc[0]) * (1 / frame_rate)
    swim_distance = subset_df['distance_since_last_frame'].sum()
    mean_swim_velocity = subset_df['Speed'].mean()
    max_swim_velocity = subset_df['Speed'].max()

    # Create the metrics dataframe
    metrics_list = [['swim duration', swim_duration], ['swim distance', swim_distance], ['mean swim velocity', mean_swim_velocity], ['max swim velocity', max_swim_velocity]]
    metrics_df = pd.DataFrame(metrics_list)

    # Format the dataframe
    df_formatted = subset_df[['Frame Number', 'X', 'Y', 'X_norm', 'Y_norm', 'distance_since_last_frame', 'Speed']]

    # Combine the dataframes manually because .concat refuses to work
    df_formatted = df_formatted.reset_index(drop=True)
    metrics_df = metrics_df.reset_index(drop=True)

    # Determine the maximum number of rows needed
    max_len = max(len(df_formatted), len(metrics_df))

    # Create an empty dataframe with the appropriate columns
    df_out = pd.DataFrame(index=range(max_len), columns=df_formatted.columns.tolist() + metrics_df.columns.tolist())

    # Fill the new dataframe with values from df_formatted and metrics_df
    df_out[df_formatted.columns] = df_formatted
    df_out[metrics_df.columns] = metrics_df

    df_out.to_csv(new_csv_path, index=False)

def extract_metrics_from_directory(xy_cal, frame_rate, directory):
    """ Calls the extract_metrics_from_predictions function on all CSVs in a directory.

    Args:
      xy_cal (float): Calibration factor to convert pixel values to real space.
      frame_rate (int): Specifies the sampling rate so that swim duration and speed can be evaluated
      directory (str): The directory containing the corrected predicted tracking data csvs.
    """
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    vid_num=1
    for file in csv_files:
        extract_metrics_from_predictions(xy_cal, frame_rate, file, vid_num)
        vid_num+=1

def format_csv_for_imagej(df, directory, output_csv):
    """ Arranges corrected and formatted tracking data so that it can be read into ImageJ

    Takes one formatted_data CSV file as input, and arranges the columns using pandas such that the output CSV can be read into
    ImageJ's manual tracking plugin along with its associated annotated video file.

    Args:
      df (pd.Dataframe): The dataframe containing the corrected, predicted tracking data for a single fish
      directory (str): The directory in which the output csvs should be stored
      output_csv (str): The name of the csv file which should be created
    """
    # Initialize the output dataframe with the required columns
    output_df = pd.DataFrame()

    # Fill the empty header column with sequential numbers starting from 1
    output_df[''] = range(1, len(df) + 1)

    # Fill the 'Track n°' column with 1s
    output_df['Track n°'] = 1

    # Copy the 'Slice n°' (Frame Number) from the input dataframe
    output_df['Slice n°'] = df['Frame Number']

    # Copy the X and Y data from the input dataframe
    output_df['X'] = df['X']
    output_df['Y'] = df['Y']

    # Set the 'Distance', 'Velocity', and 'Pixel Value' columns to 0
    output_df['Distance'] = 0
    output_df['Velocity'] = 0
    output_df['Pixel Value'] = 0

    # Ensure the columns are in the correct order
    output_df = output_df[['', 'Track n°', 'Slice n°', 'X', 'Y', 'Distance', 'Velocity', 'Pixel Value']]

    # Write the output dataframe to a CSV file
    output_dir = directory + '/predictions'
    os.makedirs(output_dir, exist_ok=True)

    output_txt = output_csv.split('.csv')[0] + '.txt'

    output_filepath = os.path.join(output_dir, output_txt)
    output_df.to_csv(output_filepath, sep='\t', index=False)


def combine_csv_to_xlsx(video_directory, directory):
    """ Combines all CSVs in a directory into one xlsx file,

    Outputs an xlsx file which contains two sheets: one with all the fish data, and the other a summary page

    Args:
      directory (str): File path to a directory which contains only csvs of formatted, corrected, predicted tracking data.

    Returns:
      output_path (str): The path to the created xlsx file
    """
    #Create the filepath for the output file
    output_file = os.path.split(video_directory)[-1] + '_results.xlsx'
    output_path = os.path.join(video_directory, output_file)

    # Create a writer object to write to Excel file
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        # Initialize a variable to keep track of the current row in the 'Fish Data' sheet
        #current_row = 0

        # Create empty DataFrame for Data Overview
        data_overview = pd.DataFrame(columns=['Fish Number', 'Swim Duration', 'Swim Distance', 'Mean Swim Velocity', 'Max Swim Velocity'])

        # Initialize lists to store data for calculations
        col9_1_list, col9_2_list, col9_3_list, col9_4_list = [], [], [], []

        fish_number = 1

        # Define and use a natural sort function to turn the filename list into human readable form (i.e. csvs listed in order)
        def natural_sort_key(s):
            return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

        filenames = [filename for filename in os.listdir(directory) if filename.endswith(".csv")]
        filenames.sort(key=natural_sort_key)

        for filename in filenames:
            if filename.endswith(".csv"):
                # Read the CSV file
                csv_path = os.path.join(directory, filename)
                df = pd.read_csv(csv_path)

                # Check if the CSV file has at least 9 columns
                if df.shape[1] < 9:
                    print(f"Skipping {filename} as it has fewer than 9 columns")
                    continue

                # Create a CSV that can be opened in ImageJ, for review
                format_csv_for_imagej(df, video_directory, filename)

                # Create a new DataFrame with the required structure
                fish_df = pd.DataFrame(columns=['Fish Number'] + df.columns.tolist() + ['Blank'])
                fish_df['Fish Number'] = ['Fish {}'.format(fish_number)] + [''] * (len(df) - 1)
                fish_df[df.columns] = df
                fish_df['Blank'] = ''

                # Determine the starting column for the current fish data
                startcol = (fish_number - 1) * 11

                # Append the fish_df to the Excel writer
                fish_df.to_excel(writer, sheet_name='Fish Data', startrow=0, startcol=startcol, index=False, header=True)

                # Update the current row
                #current_row += len(fish_df) + 1  # +1 for the header row

                # Extract the required data for Data Overview
                col9_data = df.iloc[:, 8]
                col9_1_list.append(col9_data.iloc[0])
                col9_2_list.append(col9_data.iloc[1])
                col9_3_list.append(col9_data.iloc[2])
                col9_4_list.append(col9_data.iloc[3])

                fish_number += 1

        # Construct the Data Overview DataFrame
        data_overview['Fish Number'] = ['Fish {}'.format(i+1) for i in range(len(col9_1_list))]
        data_overview['Swim Duration'] = col9_1_list
        data_overview['Swim Distance'] = col9_2_list
        data_overview['Mean Swim Velocity'] = col9_3_list
        data_overview['Max Swim Velocity'] = col9_4_list

        # Create the labels and values DataFrame
        labels_values = pd.DataFrame({
            'Label': ['average swim duration (s)', 'average swim distance (mm)', 'average mean swim velocity (mm/s)', 'average max swim velocity (mm/s)'],
            'Value': [data_overview['Swim Duration'].mean(), data_overview['Swim Distance'].mean(), data_overview['Mean Swim Velocity'].mean(), data_overview['Max Swim Velocity'].mean()]
        })

        # Write the Data Overview and labels/values to the second sheet
        data_overview.to_excel(writer, sheet_name='Data Overview', index=False)
        labels_values.to_excel(writer, sheet_name='Data Overview', startrow=len(data_overview) + 2, index=False)

    return output_path

def update_excel_with_formulas(file_path, frame_rate):
    """ Updates an xlsx file with formulas in place of values.

    Updates an existing Excel file to replace hardcoded swim metric values
    with formulas in both the 'Fish Data' and 'Data Overview' sheets.

    Args:
      file_path (str): The path to the xlsx file
      frame_rate (int): Specifies the sampling rate so that swim duration and speed can be evaluated
    """
    wb = load_workbook(file_path)
    fish_ws = wb["Fish Data"]
    overview_ws = wb["Data Overview"]

    # Detect number of fish based on column pattern (11 columns per fish)
    max_col = fish_ws.max_column
    fish_count = max_col // 11

    for fish_index in range(fish_count):
        base_col = fish_index * 11
        col_Dist = base_col + 7  # 0-indexed, distance column (column 7)
        col_Speed = base_col + 8  # speed column (column 8)
        col_Metrics = base_col + 10  # column 10: where formulas go

        # Determine data start and end rows for distance/speed
        start_row = 2  # assuming row 1 is header, data starts on row 2
        end_row = fish_ws.max_row

        # Get column letters
        dist_col_letter = get_column_letter(col_Dist)
        speed_col_letter = get_column_letter(col_Speed)
        metric_col_letter = get_column_letter(col_Metrics)

        # Metrics labels are assumed to be at rows 1–4 (1-indexed), metrics at col 10
        # 1. Swim duration (count of distance values)
        fish_ws[f"{metric_col_letter}2"].value = f'=(COUNT({dist_col_letter}{start_row}:{dist_col_letter}{end_row})-1)/{frame_rate}'
        # 2. Swim distance
        fish_ws[f"{metric_col_letter}3"].value = f'=SUM({dist_col_letter}{start_row}:{dist_col_letter}{end_row})'
        # 3. Mean swim speed
        fish_ws[f"{metric_col_letter}4"].value = f'=AVERAGE({speed_col_letter}{start_row}:{speed_col_letter}{end_row})'
        # 4. Max swim speed
        fish_ws[f"{metric_col_letter}5"].value = f'=MAX({speed_col_letter}{start_row}:{speed_col_letter}{end_row})'

    # Update "Data Overview" sheet
    for fish_index in range(fish_count):
        row = fish_index + 2  # Row 2 onward in overview sheet
        base_col = fish_index * 11
        metrics_col_letter = get_column_letter(base_col + 10)

        for metric_offset in range(4):
            fish_metric_cell = f"'Fish Data'!{metrics_col_letter}{metric_offset + 2}"
            overview_ws.cell(row=row, column=metric_offset + 2).value = f"={fish_metric_cell}"

    # Write average formulas in overview sheet column 7
    metrics = ["Swim Duration", "Swim Distance", "Mean Swim Speed", "Max Swim Speed"]
    for i, label in enumerate(metrics):
        overview_ws.cell(row=i + 2, column=7).value = f"Average {label}"
        col_letter = get_column_letter(i + 2)
        overview_ws.cell(row=i + 2, column=8).value = f"=AVERAGE({col_letter}2:{col_letter}{fish_count + 1})"

    # Save workbook
    wb.save(file_path)

def get_video_parameters():
    while True:
        try:
            xy_calibration = float(input("What is the xy calibration value? "))
            if 0 < xy_calibration < 1:
                break
            else:
                print("Please enter a number greater than 0 and less than 1.")
        except ValueError:
            print("Invalid input. Please enter a floating point number.")

    while True:
        frame_rate_input = input("What is the frame rate of your videos? (Press Enter for default = 30) ")
        if frame_rate_input.strip() == "":
            frame_rate = 30
            break
        try:
            frame_rate = int(frame_rate_input)
            if frame_rate >= 30:
                break
            else:
                print("Frame rate must be at least 30.")
        except ValueError:
            print("Invalid input. Please enter an integer or press Enter for the default.")

    return xy_calibration, frame_rate

def videos_ready():
    folder_path = "drive/MyDrive/Colab Notebooks/ZebraTrack/videos"

    # Check if folder exists
    if not os.path.exists(folder_path):
        print("The required video folder does not exist. Please ensure a folder exists at:", folder_path)
        return False

    files = os.listdir(folder_path)

    if not files:
        print("The video folder exists but is empty. Please retry after uploading videos to:", folder_path)
        return False

    avi_files = [f for f in files if f.lower().endswith(".avi")]
    non_avi_files = [f for f in files if not f.lower().endswith(".avi")]

    if avi_files and not non_avi_files:
        return True

    if non_avi_files:
        print("The folder contains files that are not .avi videos:")
        for f in non_avi_files:
            print(" -", f)

        while True:
            choice = input("Would you like to clear the folder and try again after uploading videos? (y/n): ").strip().lower()
            if choice == "y":
                for f in files:
                    path = os.path.join(folder_path, f)
                    try:
                        if os.path.isfile(path):
                            os.remove(path)
                        elif os.path.isdir(path):
                            shutil.rmtree(path)
                    except Exception as e:
                        print(f"Error removing {f}: {e}")
                print("Folder has been cleared.")
                return False
            elif choice == "n":
                print("No changes made to the folder.")
                return False
            else:
                print("Invalid input. Please enter 'y' for yes or 'n' for no.")

    if not avi_files:
        print("No .avi files found in the folder.")
        return False

def zebratrack(video_directory='drive/MyDrive/Colab Notebooks/ZebraTrack/videos'):
    """ Overarching function which creates an xlsx file of tracking results from a video directory

    Calls helper functions to direct the extraction of predictions (using YOLOv8), correction of these predictions,
    and finally the extraction of data from these predictions.

    Args:
      video_directory (str): Path to a directory which contains a set of 2-day-old larval touch response videos. The videos must be in 640x480 resolution.
      xy_cal (float): Calibration factor to convert pixel values to real space.
      frame_rate (int): Specifies the sampling rate so that swim duration and speed can be evaluated. Sampling rate must be >= 30 hz.

    Example usage:
      >>> MASTER_VIDEO_DIRECTORY = 'drive/MyDrive/Colab Notebooks/Zebrafish Data/TDP43 G348C Touch Response 02-08-24/WT'
      >>> videos_to_fish_data(MASTER_VIDEO_DIRECTORY, 0.337408, 30)
    """
    # Verify the existence of the videos folder
    if not videos_ready():
        print("\n!!!--- The program was terminated prematurely due to an absent or incorrectly formatted video folder. Please try again ---!!!")
        return

    # Obtain user input for xy_cal, frame_rate
    xy_cal, frame_rate = get_video_parameters()

    # First, extract raw predictions from the videos in the directory using YOLOv8
    videos_to_predictions(video_directory, frame_rate, False)
    print("!!!--- videos_to_predictions ran successfully ---!!!")

    # Second, correct the raw predicted_csvs, for double detections, missed detections, etc.
    csv_directory = os.path.join(video_directory, 'predicted_csvs')
    correct_csv_files(csv_directory, xy_cal)
    print("!!!--- correct_csv_files ran successfully ---!!!")

    # Third, extract the useful frames only (those in which the fish is swimming) and format the CSVs further
    corrected_directory = os.path.join(csv_directory, 'corrected_csvs')
    extract_metrics_from_directory(xy_cal, frame_rate, corrected_directory)
    print("!!!--- extract_metrics_from_directory ran successfully ---!!!")

    # Fourth, create an excel file by collating each fish's CSV to an xlsx file, and create an overview sheet
    formatted_csv_directory = os.path.join(corrected_directory, 'formatted_data')
    file_path = combine_csv_to_xlsx(video_directory, formatted_csv_directory)
    print("!!!--- combine_csv_to_xlsx ran successfully ---!!!")

    # Fifth, edit the excel file so that excel formulas are present instead of hard-coded values
    update_excel_with_formulas(file_path, frame_rate)
    print("!!!--- update_excel_with_formulas ran succesfully ---!!!")

    # Sixth and finally, remove temporary files # !!!~--- REPLACE WITH NON-GOOGLE PATH ---~!!!
    shutil.rmtree(f'drive/MyDrive/Colab Notebooks/ZebraTrack/videos/predicted_csvs')



Mounted at /content/drive


# Run ZebraTrack

Once you have followed all the setup instructions above, press play on the following cell to start!

In [24]:
zebratrack()

The required video folder does not exist. Please ensure a folder exists at: drive/MyDrive/Colab Notebooks/ZebraTrack/videos

!!!--- The program was terminated prematurely due to an absent or incorrectly formatted video folder. Please try again ---!!!
